# Training

Experiments were conducted using:
- `mb_vae_dti/training/run.py`: command line interface for training
- and the scripts in `scripts/training/` for running the experiments

This notebook shows some plots and analysis of the unsupervised pretraining, general DTI training and benchmark fine-tuning results.

In [57]:
from resolve import *

In [190]:
from mb_vae_dti.validating.analysis import *

df = load_gridsearch_results("notebooks/results/mo_cold/")

if "metadata.data.provenance_cols" in df.columns:
    df["metadata.data.provenance_cols"] = df["metadata.data.provenance_cols"].apply(lambda x: x[0])
if "metadata.data.drug_features" in df.columns:
    df = df.drop(columns=["metadata.data.drug_features", "metadata.data.target_features"])

df.sort_values(by="best_val_loss", ascending=True)

2025-07-29 13:56:29,796 - mb_vae_dti.validating.analysis - INFO - Successfully loaded 54 results with 10 columns


,experiment_name,best_val_loss,best_epoch,val_metrics,test_metrics,trainable_params,config,timing.total_training_time,timing.avg_time_per_epoch,timing.total_epochs
16,multi_output_train_cold_b03c0003,0.930683,23,"{'val/loss_Y': 0.3950285017490387, 'val/loss_Y...","{'test/loss_Y': 0.41485193371772766, 'test/los...",44318469,"{'training.learning_rate': 0.001, 'training.sc...",22617.284016,628.257889,36
40,multi_output_train_cold_b04c0004,0.949205,23,"{'val/loss_Y': 0.4063224196434021, 'val/loss_Y...","{'test/loss_Y': 0.4226910173892975, 'test/loss...",44318469,"{'training.learning_rate': 0.0005, 'training.s...",23325.871034,647.940862,36
25,multi_output_train_cold_b05c0006,0.976269,15,"{'val/loss_Y': 0.39521554112434387, 'val/loss_...","{'test/loss_Y': 0.39312589168548584, 'test/los...",45114885,"{'training.learning_rate': 0.001, 'training.sc...",19861.862970,709.352249,28
39,multi_output_train_cold_b04c0000,0.978312,30,"{'val/loss_Y': 0.424538791179657, 'val/loss_Y_...","{'test/loss_Y': 0.4308117628097534, 'test/loss...",45114885,"{'training.learning_rate': 0.0005, 'training.s...",32525.072075,756.397025,43
19,multi_output_train_cold_b04c0002,1.010602,23,"{'val/loss_Y': 0.378013014793396, 'val/loss_Y_...","{'test/loss_Y': 0.3944063186645508, 'test/loss...",44318469,"{'training.learning_rate': 0.0005, 'training.s...",23376.830587,649.356405,36
30,multi_output_train_cold_b03c0005,1.031901,18,"{'val/loss_Y': 0.39519548416137695, 'val/loss_...","{'test/loss_Y': 0.39965787529945374, 'test/los...",44318469,"{'training.learning_rate': 0.001, 'training.sc...",20574.612862,663.697189,31
13,multi_output_train_cold_b01c0000,1.055216,30,"{'val/loss_Y': 0.413931667804718, 'val/loss_Y_...","{'test/loss_Y': 0.42732730507850647, 'test/los...",45114885,"{'training.learning_rate': 0.0005, 'training.s...",32669.414825,759.753833,43
22,multi_output_train_cold_b01c0006,1.065606,33,"{'val/loss_Y': 0.3943135738372803, 'val/loss_Y...","{'test/loss_Y': 0.4135196805000305, 'test/loss...",44318469,"{'training.learning_rate': 0.001, 'training.sc...",15363.062677,333.979623,46
12,multi_output_train_cold_b01c0004,1.069636,14,"{'val/loss_Y': 0.3764750063419342, 'val/loss_Y...","{'test/loss_Y': 0.3752148747444153, 'test/loss...",44318469,"{'training.learning_rate': 0.0001, 'training.s...",17504.269518,648.306278,27
21,multi_output_train_cold_b03c0008,1.076028,30,"{'val/loss_Y': 0.4246678650379181, 'val/loss_Y...","{'test/loss_Y': 0.4334804117679596, 'test/loss...",45114885,"{'training.learning_rate': 0.0001, 'training.s...",31228.618726,726.246947,43


In [159]:
varying_attrs = get_varying_attributes(df)
varying_attrs.remove('metadata.logging.experiment_name')
varying_attrs.extend(["best_val_loss", 'best_epoch', 'trainable_params', "timing.total_training_time"])
varying_attrs

2025-07-27 19:35:42,354 - mb_vae_dti.validating.analysis - INFO - Found 8 varying attributes: ['metadata.training.learning_rate', 'metadata.data.batch_size', 'metadata.logging.experiment_name', 'metadata.model.encoder_kwargs.hidden_dim', 'metadata.model.encoder_kwargs.n_layers', 'metadata.model.embedding_dim', 'metadata.model.encoder_type', 'metadata.model.aggregator_type']


['metadata.training.learning_rate',
 'metadata.data.batch_size',
 'metadata.model.encoder_kwargs.hidden_dim',
 'metadata.model.encoder_kwargs.n_layers',
 'metadata.model.embedding_dim',
 'metadata.model.encoder_type',
 'metadata.model.aggregator_type',
 'best_val_loss',
 'best_epoch',
 'trainable_params',
 'timing.total_training_time']

In [160]:
subdf = get_top_performers(df, n_top=5)[varying_attrs]
# for each columns, rename the column by splitting the column name by "." and taking the last element
subdf.columns = subdf.columns.str.split(".").str[-1]
print(f"Total training time: {(subdf['total_training_time'].sum() / 3600):.2f} hours")
print(f"Average trainable params: {subdf['trainable_params'].mean() / 1000000:.1f}M")
subdf

Total training time: 7.57 hours
Average trainable params: 10.1M


,learning_rate,batch_size,hidden_dim,n_layers,embedding_dim,encoder_type,aggregator_type,best_val_loss,best_epoch,trainable_params,total_training_time
290,0.0005,16,256,3,1024,resnet,concat,0.154958,85,10555136,7962.247344
422,0.0010,32,512,1,1024,resnet,concat,0.155305,83,16484864,2606.612936
300,0.0010,16,256,3,1024,resnet,concat,0.156679,93,10555136,8547.191034
379,0.0020,16,256,1,1024,resnet,concat,0.157262,95,5294336,4868.378374
229,0.0005,32,256,2,768,resnet,concat,0.157600,90,7596544,3258.366287


In [176]:
a = 0.65317
b = 0.62066
c = 0.64136
d = 0.6565
e = 0.64104


total = a + b + c + d + e
print(total / 5)

0.642546


In [186]:
ensemble = """
Name
5 visualized
test/Y_KIBA_ci
test/Y_KIBA_mse
test/Y_KIBA_pearson
test/Y_KIBA_r2
test/Y_KIBA_rmse
•
multi_modal_finetune_KIBA_rand_ensemble
0.85238
0.18802
0.84787
0.71881
0.43361
•
multi_modal_finetune_KIBA_rand_ensemble
0.84646
0.19966
0.83758
0.70141
0.44683
•
multi_modal_finetune_KIBA_rand_ensemble
0.84461
0.20252
0.83521
0.69713
0.45002
•
multi_modal_finetune_KIBA_rand_ensemble
0.84727
0.19507
0.84162
0.70826
0.44167
•
multi_modal_finetune_KIBA_rand_ensemble
0.84913
0.19269
0.84389
0.71183
0.43896
"""
from statistics import mean
lines = [line.strip() for line in ensemble.strip().split('\n') if line.strip()]
first_bullet = next(i for i, line in enumerate(lines) if line == '•')
start_idx = 2
column_names = lines[start_idx:first_bullet]
data = {col: [] for col in column_names}

i = first_bullet + 1
while i < len(lines):
    line = lines[i]
    if line == '•' or line.startswith('multi_modal_'):
        i += 1
        continue
    for j, col_name in enumerate(column_names):
        if i + j < len(lines):
            try:
                value = float(lines[i + j])
                data[col_name].append(value)
            except ValueError:
                break
    
    i += len(column_names)

results = {}
for col_name, values in data.items():
    results[col_name] = mean(values)

results

{'test/Y_KIBA_ci': 0.84797,
 'test/Y_KIBA_mse': 0.195592,
 'test/Y_KIBA_pearson': 0.841234,
 'test/Y_KIBA_r2': 0.707488,
 'test/Y_KIBA_rmse': 0.442218}

In [187]:
data

{'test/Y_KIBA_ci': [0.85238, 0.84646, 0.84461, 0.84727, 0.84913],
 'test/Y_KIBA_mse': [0.18802, 0.19966, 0.20252, 0.19507, 0.19269],
 'test/Y_KIBA_pearson': [0.84787, 0.83758, 0.83521, 0.84162, 0.84389],
 'test/Y_KIBA_r2': [0.71881, 0.70141, 0.69713, 0.70826, 0.71183],
 'test/Y_KIBA_rmse': [0.43361, 0.44683, 0.45002, 0.44167, 0.43896]}